In [1]:
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import cvxpy as cp
import scipy
from networkx.algorithms.flow import edmonds_karp

## Exercise 3

In [2]:
f = scipy.io.loadmat('flow.mat')["flow"].reshape(28,)
C = scipy.io.loadmat('capacities.mat')["capacities"].reshape(28,)
B = scipy.io.loadmat('traffic.mat')["traffic"]
l = scipy.io.loadmat('traveltime.mat')["traveltime"].reshape(28,)

In [3]:
HG = nx.DiGraph()

num_nodes, num_edges = B.shape

# aggiungiamo nodi
HG.add_nodes_from(range(num_nodes))

# ricaviamo archi e capacità
for j in range(num_edges):
    src = np.where(B[:, j] == 1)[0][0]
    dst = np.where(B[:, j] == -1)[0][0]
    HG.add_edge(src, dst, capacity=int(C[j]), travel_time=l[j])

In [4]:
def check_non_negative_edges(G):
    for u, v, data in G.edges(data=True):
        if G[u][v]['travel_time'] < 0:
            return False
    return True

In [5]:
check_non_negative_edges(HG)

True

In [6]:
#nx.draw(HG)

### a) Find the shortest path between node 1 and 17. This is equivalent to the fastest path (path with shortest traveling time) in an empty network.

In [7]:
def find_shortest_path_cvxpy(G, source, dest):
    
    H = G

    x = {}
    for u, v in H.edges():
        # 1 if we use (u,v), 0 otherwise
        x[u, v] = cp.Variable(boolean=True) 

    # the objective function to minimize is the sum of the travel time of the edges used
    cost = 0
    for u, v, data in H.edges(data=True):
        weight = data.get('travel_time')
        cost += x[u, v] * weight
        
    objective = cp.Minimize(cost)

    constraints = []
    for node in H.nodes():
        flow_in = cp.sum([x[u, v] for u, v in H.in_edges(node)])
        
        flow_out = cp.sum([x[u, v] for u, v in H.out_edges(node)])

        # flow conservation constraints
        if node == source:
            # source has one out edge
            constraints.append(flow_out - flow_in == 1)
        elif node == dest:
            # dest has one in edge
            constraints.append(flow_in - flow_out == 1)
        else:
            # intermediate nodes have equal in and out 
            constraints.append(flow_in - flow_out == 0)

    problem = cp.Problem(objective, constraints)
     
    problem.solve() 


    total_cost = problem.value
    
    # build the shortest path
    # pick edges where x[u,v] == 1
    path = [source]
    current_node = source
    
    while current_node != dest:
        for u, v in H.edges():
            # Check if the edge starts from the current node
            if u == current_node and x[u, v].value is not None:
                path.append(v)
                current_node = v
                break

    return path, total_cost


print("CVXPY solution")
path_cvxpy, cost_cvxpy = find_shortest_path_cvxpy(HG, 0, 16)
if path_cvxpy:
    print("Path: " + " -> ".join([str(node + 1) for node in path_cvxpy]))
    print(f"Total travel time: {cost_cvxpy}")


print("NetworkX solution (Verify)")
path_nx = nx.shortest_path(HG, source=0, target=16, weight='travel_time')
cost_nx = nx.shortest_path_length(HG, source=0, target=16, weight='travel_time')
print("Path: " + " -> ".join([str(node + 1) for node in path_nx]))
print(f"Total travel time: {cost_nx}")

CVXPY solution
Path: 1 -> 2 -> 3 -> 4 -> 5 -> 14 -> 17
Total travel time: 0.559833
NetworkX solution (Verify)
Path: 1 -> 2 -> 3 -> 9 -> 13 -> 17
Total travel time: 0.559833


### b) Find the maximum flow between node 1 and 17.

In [8]:
mf = nx.flow.maximum_flow(HG, 0, 16, capacity='capacity', flow_func=edmonds_karp)
print("Maximum flow value from node 1 to node 17:", mf[0])

Maximum flow value from node 1 to node 17: 22448


### c) Given the flow vector in flow.mat, compute the vector $ν$ satisfying $Bf = ν$

In [9]:
nu = B @ f
print("Vector ν satisfying Bf = ν:", nu)

Vector ν satisfying Bf = ν: [ 16806   8570  19448   4957   -746   4768    413     -2  -5671   1169
     -5  -7131   -380  -7412  -7810  -3430 -23544]


In [10]:
nu.sum() == 0

np.True_

### d) Find the social optimum $f^∗$ with respect to the delays on the different links $τ_e(f_e)$. For this, minimize the cost function $$\sum_{e \in \mathcal E}f_e \tau_e (f(e)) = \sum_{e \in \mathcal E} \frac {f_e l_e}{1 - f_e / c_e} = \sum_{e \in \mathcal E}(\frac {l_ec_e}{1-f_e/c_e}-l_ec_e)$$ subject to the flow constraints

In [11]:
def compute_total_delay(f, func='regular'):
    """
    Computes the total delay based on the given flow `f`.
    :param f: Flow vector.
    :param func: Specifies the type of delay computation.
    :return: Total delay.
    """
    if func == 'regular':
        return np.sum(l * C / (1 - f / C) - l * C)
    elif func == 'new':
        return np.sum(l * C / (1 - f / C) - l * C + f * C)

In [12]:
n_edges = B.shape[1]
nu_source = nu[0]
nu_sink = -nu_source
nu = np.zeros(B.shape[0])
nu[0] = nu_source
nu[-1] = nu_sink
f_opt = cp.Variable(n_edges)
# use right cost expression to respect DCP rules
num = l * C
#den = 1 - f_opt / C
den = 1 - cp.multiply(f_opt, cp.inv_pos(C))
inv_den  = cp.inv_pos(den)
term1 = cp.multiply(num, inv_den)
cost = term1 - num
#print("cost function is convex?", cp.sum(cost).is_convex())
objective = cp.Minimize(cp.sum(cost))
#objective = cp.Minimize(cp.sum(cp.multiply(l * C, cp.inv_pos(1 - f_opt / C)) - l * C))
constraints = [B @ f_opt == nu, f_opt >=0, f_opt <= C]
prob = cp.Problem(objective, constraints)
result = prob.solve()
print("Optimal flow values on edges:", f_opt.value)

Optimal flow values on edges: [6.56934343e+03 5.81000870e+03 3.04697202e+03 3.04697044e+03
 1.02366566e+04 4.66633524e+03 3.06121109e+03 2.59598535e+03
 3.10455509e+03 7.59334729e+02 5.72744560e-03 2.76303096e+03
 1.57821332e-03 3.04697044e+03 5.57032133e+03 2.89383855e+03
 5.04094165e+03 2.36445888e+03 4.65231471e+02 2.25446280e+03
 3.35906803e+03 5.61353083e+03 2.37198823e+03 1.99673781e-03
 6.34609768e+03 5.41895867e+03 5.04094365e+03 5.04094365e+03]


In [13]:
prob.solver_stats.solver_name

'CLARABEL'

In [14]:
total_delay = compute_total_delay(f_opt.value, func='regular')
print("Total delay with optimal flow:", total_delay, result)

Total delay with optimal flow: 26142.669749995475 26142.669749995475


### e) Find the Wardrop equilibrium $f^{(0)}$. For this, use the cost function $$\psi_e(f_e) = \sum_{e \in \mathcal E} \int_{0}^{f_e} \tau_e(s)ds = -\sum_{e \in \mathcal E} l_ec_e \text{ln}(1 - \frac {f_e}{c_e})$$

In [15]:
f_w = cp.Variable(n_edges)
objective_w = cp.sum(cp.multiply(-l * C, cp.log(1 - cp.multiply(f_w, cp.inv_pos(C)))))
constraints_w = [B @ f_w == nu, f_w >=0, f_w <= C]
prob_w = cp.Problem(cp.Minimize(objective_w), constraints_w)
f_w_opt = prob_w.solve()
print("Wardrop equilibrium:", f_w.value)

Wardrop equilibrium: [6.55750561e+03 6.30859527e+03 2.20072140e+03 2.20072138e+03
 1.02484944e+04 4.70669121e+03 2.85981287e+03 2.23263877e+03
 3.34992731e+03 2.48910340e+02 1.16462355e+01 4.09622764e+03
 1.44395841e-05 2.20072138e+03 5.54180318e+03 2.34344230e+03
 5.29414955e+03 2.09578867e+03 6.38820341e+02 2.97893911e+03
 2.98226262e+03 5.96120173e+03 2.52236088e+03 2.24569380e-05
 6.78876816e+03 4.72308226e+03 5.29414957e+03 5.29414957e+03]


In [16]:
total_delay_we = compute_total_delay(f_w.value)
print("Total delay at Wardrop equilibrium:", total_delay_we, f_w_opt)

Total delay at Wardrop equilibrium: 26495.287958196597 15731.952643030687


In [17]:
total_delay_we / total_delay

np.float64(1.013488224866597)

### f) Introduce tolls $$\frac{df_e}{​dτ_e}​​=\frac{l_e}{c_e​(1−f_e​/c_e​)^2}​​$$

In [18]:
f_toll = cp.Variable(n_edges)
den_toll = cp.multiply(C, cp.power(1 - f_opt.value / C, 2)) # c * (1 - f / C)^2
inv_den_toll = cp.inv_pos(den_toll)
delay_der = cp.multiply(l, inv_den_toll)
toll = cp.multiply(f_opt.value, delay_der)
log_term_toll = cp.log(1 - f_toll / C)
objective_toll = cp.sum(cp.multiply(-l *C, log_term_toll))
constraints_toll = [B @ f_toll == nu, f_toll >=0, f_toll <= C]
prob_toll = cp.Problem(cp.Minimize(objective_toll + toll.T @ f_toll), constraints_toll)
res_toll = prob_toll.solve()
print("Wardrop equilibrium with tolls:", f_toll.value)

Wardrop equilibrium with tolls: [6.56815120e+03 5.80995135e+03 3.04697769e+03 3.04697768e+03
 1.02378488e+04 4.66745753e+03 3.06125944e+03 2.59585166e+03
 3.10451734e+03 7.58199850e+02 3.92407823e-05 2.76297362e+03
 1.43200202e-05 3.04697768e+03 5.57039127e+03 2.89380129e+03
 5.04098792e+03 2.36439794e+03 4.65407815e+02 2.25430795e+03
 3.35920908e+03 5.61351703e+03 2.37195384e+03 2.11646894e-05
 6.34608053e+03 5.41893152e+03 5.04098794e+03 5.04098794e+03]


In [19]:
#check if wardrop equilibrium with tolls is equal with optimum flow (lab 3)
f_opt.value - f_toll.value

array([ 1.19223368,  0.05735491, -0.0056752 , -0.0072391 , -1.19222669,
       -1.1222843 , -0.04834243,  0.13368997,  0.03774927,  1.13487877,
        0.0056882 ,  0.05734191,  0.00156389, -0.0072391 , -0.06994238,
        0.03726615, -0.04627164,  0.06093689, -0.17634419,  0.1548465 ,
       -0.14105362,  0.01379288,  0.03438926,  0.00197557,  0.01715289,
        0.02715016, -0.04429606, -0.04429606])

In [20]:
total_delay_tolls = compute_total_delay(f_toll.value)
print("Total delay with tolls at optimal flow:", total_delay_tolls)

Total delay with tolls at optimal flow: 26142.67135556477


In [21]:
total_delay_tolls / total_delay

np.float64(1.0000000614156592)